In [40]:
import numpy as np
import pandas as pd

In [41]:
df = pd.read_csv('https://raw.githubusercontent.com/shekharbiswas/MHC/main/mental-heath-in-tech-2016_20161114.csv')
df

,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",...,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your age?,What is your gender?,What country do you live in?,What US state or territory do you live in?,What country do you work in?,What US state or territory do you work in?,Which of the following best describes your work position?,Do you work remotely?
0,0,26-100,1.0,NaN,Not eligible for coverage / N/A,NaN,No,No,I don't know,Very easy,...,Not applicable to me,Not applicable to me,39,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Sometimes
1,0,Jun-25,1.0,NaN,No,Yes,Yes,Yes,Yes,Somewhat easy,...,Rarely,Sometimes,29,male,United States of America,Illinois,United States of America,Illinois,Back-end Developer|Front-end Developer,Never
2,0,Jun-25,1.0,NaN,No,NaN,No,No,I don't know,Neither easy nor difficult,...,Not applicable to me,Not applicable to me,38,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Always
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Sometimes,Sometimes,43,male,United Kingdom,NaN,United Kingdom,NaN,Supervisor/Team Lead,Sometimes
4,0,Jun-25,0.0,1.0,Yes,Yes,No,No,No,Neither easy nor difficult,...,Sometimes,Sometimes,43,Female,United States of America,Illinois,United States of America,Illinois,Executive Leadership|Supervisor/Team Lead|Dev ...,Sometimes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Not applicable to me,Not applicable to me,34,Female,United States of America,New York,United States of America,New York,Other,Sometimes
1429,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Sometimes,Often,56,MALE,United States of America,California,Afghanistan,NaN,Support,Sometimes
1430,0,100-500,1.0,NaN,Yes,Yes,Yes,Yes,I don't know,Somewhat difficult,...,Rarely,Sometimes,52,Male,United States of America,Georgia,United States of America,Georgia,Back-end Developer,Sometimes
1431,0,100-500,0.0,1.0,I don't know,I am not sure,No,Yes,I don't know,Somewhat difficult,...,Sometimes,Often,30,Female,United States of America,Nebraska,United States of America,Nebraska,DevOps/SysAdmin,Sometimes


In [58]:
old_names = df.columns # Just to be sure to be able to look that up
renamed_columns = ['self_employed', 'number_of_employees', 'company_in_tech', 'role_in_tech', 'company_mental_health',
                  'company_mental_health_options_awareness', 'company_mental_health_discussion', 'company_mental_health_resources', 
                   'company_mental_health_anonymity','company_mental_health_leave', 'company_mental_health_discussion_impact', 
                   'company_physical_health_discussion_negative_impact', 'mental_health_coworkers_discussion',
                   'mental_health_supervisor_discussion', 'company_mental_health_serious', 
                   'company_mental_health_coworker_discussion_negative_impact', 'mental_health_coverage', 'mental_health_online_resources',
                  'mental_health_diagnosed_reveal', 'mental_health_diagnosed_reveal_impact', 'mental_health_diagnosed_coworker_reveal', 
                   'mental_health_diagnosed_coworker_reveal_impact_negative','mental_health_productivity_impact', 
                   'mental_health_productivity_impact_percent', 'previous_companies', 'previous_companies_mental_health_benefits', 
                   'previous_companies_mental_health_benefits_awareness','previous_companies_mental_health_discussion', 
                   'previous_companies_mental_health_resources', 'previous_companies_mental_health_anonymity', 
                   'previous_companies_mental_health_discussion_negative_impact', 
                   'previous_companies_physical_health_discussion_negative_impact',
                  'previous_companies_mental_coworkers_health_discussion', 'previous_companies_mental_supervisor_health_discussion', 
                   'previous_company_mental_health_serious', 'previous_company_mental_health_coworker_discussion_negative_impact',
                  'future_company_physical_health_interview', 'why/why_not', 'future_company_mental_health_interview', 'why/why_not2', 
                   'mental_health_negative_impact_career', 'mental_health_negative_view_coworkers',
                  'mental_health_reveal_family&friends', 'previous_and_current_company_mental_health_bad_response_experienced', 
                   'previous_and_current_company_mental_health_bad_response_experienced_others', 'mental_health_family_history',
                  'mental_health_disorder_past', 'mental_health_disorder_current', 'yes:what_diagnosis', 'maybe:what_diagnosis', 
                   'mental_health_disorder_professional_diagnosis','yes:condition_diagnosed', 'mental_health_proffesional_treatment', 
                   'mental_health_treatment_affects_work', 'mental_health_no_treatment_affects_work','age', 'sex', 'country_live', 
                   'live_us_territory', 'country_work', 'work_us_territory', 'work_position', 'remote']
    
df.columns = renamed_columns

In [59]:
unclear_data_columns = []
for column in df.columns:
    if len(df[column].value_counts(dropna=False)) > 10: # If the column has more than 10 unique entries, the column might be unqualified for being used in our analysis
        unclear_data_columns.append(column)

        
for column in unclear_data_columns:
    print(column)

why/why_not
why/why_not2
yes:what_diagnosis
maybe:what_diagnosis
yes:condition_diagnosed
age
sex
country_live
live_us_territory
country_work
work_us_territory
work_position


In [60]:
survey_data = df.drop(["why/why_not", "why/why_not2", "yes:what_diagnosis", "maybe:what_diagnosis", "yes:condition_diagnosed", "live_us_territory", "work_us_territory", "country_live", "country_work", "work_position"], axis = 1)

In [61]:
mean_age = survey_data[(survey_data["age"] >= 18) | (survey_data["age"] <= 75)]["age"].mean()

# Replace irregular age numbers with mean value
survey_data["age"].replace(to_replace = survey_data[(survey_data["age"] < 18) | (survey_data["age"] > 75)]["age"].tolist(), value = mean_age, inplace = True)

In [62]:
survey_data["sex"] = survey_data["sex"].str.lower().str.strip()
survey_data["sex"].value_counts()

male = ['Male', 'male', 'M', 'm', 'Cis Male', 'man', 'ostensibly male, unsure what that really means', 'Mail', 'Make', 'male (cis)', 
        'cis male', 'maile', 'Malr', 'Cis Man', 'Mal', 'msle', 'male.', 'sex is male', 'malr', 'cis man', 'mail' ]
     
f = ['Female', 'female', 'F', 'f', 'Woman', 'Femake', 'Female (cis)', 'cis female', 'woman', 'femail', 
     'cis-female/femme', 'i identify as female.', 'cis-woman', 'cisgender female', 'female (props for making this a freeform field, though)', 
     'female/woman', 'female assigned at birth' ]

# Male and female
survey_data["sex"] = survey_data["sex"].replace(male, 'male')
survey_data["sex"] = survey_data["sex"].replace(f, 'f')

# Other
other = list(survey_data["sex"].value_counts().index)[2:]
survey_data["sex"] = survey_data["sex"].replace(other, 'other')

In [63]:
# We have 1433 rows with many different features and missing values, so we drop all columns where half of the overall values are missing
bad_column_numbers = (survey_data.isna().sum() >= survey_data.shape[0]/2).tolist()
bad_column_columns = survey_data.columns[bad_column_numbers]
print("Dropped columns:")
for column in bad_column_columns:
    print(column)
survey_data.drop(labels = bad_column_columns, axis = 1, inplace = True)

Dropped columns:
role_in_tech
mental_health_coverage
mental_health_online_resources
mental_health_diagnosed_reveal
mental_health_diagnosed_reveal_impact
mental_health_diagnosed_coworker_reveal
mental_health_diagnosed_coworker_reveal_impact_negative
mental_health_productivity_impact
mental_health_productivity_impact_percent
previous_and_current_company_mental_health_bad_response_experienced_others


In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
import os

In [64]:
# Replace nan values with mode
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer.fit(survey_data)
survey_data = pd.DataFrame(data = imputer.transform(survey_data), columns = survey_data.columns)

In [65]:
survey_data

,self_employed,number_of_employees,company_in_tech,company_mental_health,company_mental_health_options_awareness,company_mental_health_discussion,company_mental_health_resources,company_mental_health_anonymity,company_mental_health_leave,company_mental_health_discussion_impact,...,mental_health_family_history,mental_health_disorder_past,mental_health_disorder_current,mental_health_disorder_professional_diagnosis,mental_health_proffesional_treatment,mental_health_treatment_affects_work,mental_health_no_treatment_affects_work,age,sex,remote
0,0,26-100,1.0,Not eligible for coverage / N/A,No,No,No,I don't know,Very easy,No,...,No,Yes,No,Yes,0,Not applicable to me,Not applicable to me,39.0,male,Sometimes
1,0,Jun-25,1.0,No,Yes,Yes,Yes,Yes,Somewhat easy,No,...,Yes,Yes,Yes,Yes,1,Rarely,Sometimes,29.0,male,Never
2,0,Jun-25,1.0,No,No,No,No,I don't know,Neither easy nor difficult,Maybe,...,No,Maybe,No,No,1,Not applicable to me,Not applicable to me,38.0,male,Always
3,1,26-100,1.0,Yes,No,No,No,I don't know,Somewhat easy,Maybe,...,No,Yes,Yes,Yes,1,Sometimes,Sometimes,43.0,male,Sometimes
4,0,Jun-25,0.0,Yes,Yes,No,No,No,Neither easy nor difficult,Yes,...,Yes,Yes,Yes,Yes,1,Sometimes,Sometimes,43.0,f,Sometimes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,1,26-100,1.0,Yes,No,No,No,I don't know,Somewhat easy,Maybe,...,Yes,No,No,No,1,Not applicable to me,Not applicable to me,34.0,f,Sometimes
1429,1,26-100,1.0,Yes,No,No,No,I don't know,Somewhat easy,Maybe,...,Yes,No,No,Yes,0,Sometimes,Often,56.0,male,Sometimes
1430,0,100-500,1.0,Yes,Yes,Yes,Yes,I don't know,Somewhat difficult,Maybe,...,Yes,Yes,Maybe,Yes,1,Rarely,Sometimes,52.0,male,Sometimes
1431,0,100-500,0.0,I don't know,I am not sure,No,Yes,I don't know,Somewhat difficult,Maybe,...,Yes,Maybe,Yes,Yes,0,Sometimes,Often,30.0,f,Sometimes


In [66]:
survey_data_with_clusters = survey_data[["company_mental_health", "company_mental_health_discussion", "company_mental_health_resources",
                                         "company_mental_health_anonymity", "company_mental_health_leave", "company_mental_health_options_awareness", 
                                         "company_mental_health_discussion_impact", "mental_health_coworkers_discussion", "mental_health_supervisor_discussion", 
                                         "company_mental_health_serious", "company_mental_health_coworker_discussion_negative_impact", "previous_and_current_company_mental_health_bad_response_experienced",
                                         "mental_health_negative_impact_career", "mental_health_negative_view_coworkers","mental_health_disorder_current"]]

In [67]:
for col in survey_data_with_clusters:
    print(str(col) + " " + str(survey_data_with_clusters[col].unique()))

company_mental_health ['Not eligible for coverage / N/A' 'No' 'Yes' "I don't know"]
company_mental_health_discussion ['No' 'Yes' "I don't know"]
company_mental_health_resources ['No' 'Yes' "I don't know"]
company_mental_health_anonymity ["I don't know" 'Yes' 'No']
company_mental_health_leave ['Very easy' 'Somewhat easy' 'Neither easy nor difficult' 'Very difficult'
 'Somewhat difficult' "I don't know"]
company_mental_health_options_awareness ['No' 'Yes' 'I am not sure']
company_mental_health_discussion_impact ['No' 'Maybe' 'Yes']
mental_health_coworkers_discussion ['Maybe' 'Yes' 'No']
mental_health_supervisor_discussion ['Yes' 'Maybe' 'No']
company_mental_health_serious ["I don't know" 'Yes' 'No']
company_mental_health_coworker_discussion_negative_impact ['No' 'Yes']
previous_and_current_company_mental_health_bad_response_experienced ['No' 'Maybe/Not sure' 'Yes, I experienced' 'Yes, I observed']
mental_health_negative_impact_career ['Maybe' "No, I don't think it would" 'Yes, I think it

In [68]:
# Convert data into numerical values
# We use a rating system here 0 bad => 5 best
# Sometimes the missing knowledge of a service is equal to no service because of bad communication, so it sometimes gets the same points as having none

# company_mental_health 
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health'] == "No", 'company_mental_health'] = 0
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health'] == "Not eligible for coverage / N/A", 'company_mental_health'] = 2
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health'] == "I don't know", 'company_mental_health'] = 0 # Not knowing is equal to having none
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health'] == "Yes", 'company_mental_health'] = 5
# company_mental_health_discussion
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_discussion'] == "No", 'company_mental_health_discussion'] = 0
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_discussion'] == "I don't know", 'company_mental_health_discussion'] = 0 # Not knowing here is equal to having none
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_discussion'] == "Yes", 'company_mental_health_discussion'] = 5
# company_mental_health_resources 
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_resources'] == "No", 'company_mental_health_resources'] = 0
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_resources'] == "I don't know", 'company_mental_health_resources'] = 0 # Not knowing here is equal to having none
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_resources'] == "Yes", 'company_mental_health_resources'] = 5
# company_mental_health_anonymity 
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_anonymity'] == "No", 'company_mental_health_anonymity'] = 0
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_anonymity'] == "I don't know", 'company_mental_health_anonymity'] = 2 # This is not necessary to be communicated tho
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_anonymity'] == "Yes", 'company_mental_health_anonymity'] = 5
# company_mental_health_leave
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_leave'] == "Very easy", 'company_mental_health_leave'] = 4
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_leave'] == "Somewhat easy", 'company_mental_health_leave'] = 3
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_leave'] == "Neither easy nor difficult", 'company_mental_health_leave'] = 2
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_leave'] == "Somewhat difficult", 'company_mental_health_leave'] = 1
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_leave'] == "Very difficult", 'company_mental_health_leave'] = 0
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_leave'] == "I don't know", 'company_mental_health_leave'] = 0 # Not communicated => bad
# company_mental_health_options_awareness
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_options_awareness'] == "No", 'company_mental_health_options_awareness'] = 0
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_options_awareness'] == "I am not sure", 'company_mental_health_options_awareness'] = 2
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_options_awareness'] == "Yes", 'company_mental_health_options_awareness'] = 5
# company_mental_health_discussion_impact
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_discussion_impact'] == "No", 'company_mental_health_discussion_impact'] = 5
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_discussion_impact'] == "Maybe", 'company_mental_health_discussion_impact'] = 1 
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_discussion_impact'] == "Yes", 'company_mental_health_discussion_impact'] = 0
# mental_health_coworkers_discussion
survey_data_with_clusters.loc[survey_data_with_clusters['mental_health_coworkers_discussion'] == "No", 'mental_health_coworkers_discussion'] = 0
survey_data_with_clusters.loc[survey_data_with_clusters['mental_health_coworkers_discussion'] == "Maybe", 'mental_health_coworkers_discussion'] = 3
survey_data_with_clusters.loc[survey_data_with_clusters['mental_health_coworkers_discussion'] == "Yes", 'mental_health_coworkers_discussion'] = 5
# company_mental_health_discussion
survey_data_with_clusters.loc[survey_data_with_clusters['mental_health_supervisor_discussion'] == "No", 'mental_health_supervisor_discussion'] = 0
survey_data_with_clusters.loc[survey_data_with_clusters['mental_health_supervisor_discussion'] == "Maybe", 'mental_health_supervisor_discussion'] = 3
survey_data_with_clusters.loc[survey_data_with_clusters['mental_health_supervisor_discussion'] == "Yes", 'mental_health_supervisor_discussion'] = 5
# company_mental_health_serious
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_serious'] == "No", 'company_mental_health_serious'] = 0
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_serious'] == "I don't know", 'company_mental_health_serious'] = 2
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_serious'] == "Yes", 'company_mental_health_serious'] = 5
# company_mental_health_coworker_discussion_negative_impact
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_coworker_discussion_negative_impact'] == "No", 'company_mental_health_coworker_discussion_negative_impact'] = 5
survey_data_with_clusters.loc[survey_data_with_clusters['company_mental_health_coworker_discussion_negative_impact'] == "Yes", 'company_mental_health_coworker_discussion_negative_impact'] = 0
# previous_and_current_company_mental_health_bad_response_experienced
survey_data_with_clusters.loc[survey_data_with_clusters['previous_and_current_company_mental_health_bad_response_experienced'] == "No", 'previous_and_current_company_mental_health_bad_response_experienced'] = 5
survey_data_with_clusters.loc[survey_data_with_clusters['previous_and_current_company_mental_health_bad_response_experienced'] == "Maybe/Not sure", 'previous_and_current_company_mental_health_bad_response_experienced'] = 1 # Not knowing here is equal to having none
survey_data_with_clusters.loc[survey_data_with_clusters['previous_and_current_company_mental_health_bad_response_experienced'] == "Yes, I experienced", 'previous_and_current_company_mental_health_bad_response_experienced'] = 0
survey_data_with_clusters.loc[survey_data_with_clusters['previous_and_current_company_mental_health_bad_response_experienced'] == "Yes, I observed", 'previous_and_current_company_mental_health_bad_response_experienced'] = 0
# mental_health_negative_impact_career
survey_data_with_clusters.loc[survey_data_with_clusters['mental_health_negative_impact_career'] == "No, I don't think it would", 'mental_health_negative_impact_career'] = 5
survey_data_with_clusters.loc[survey_data_with_clusters['mental_health_negative_impact_career'] == "Maybe", 'mental_health_negative_impact_career'] = 1
survey_data_with_clusters.loc[survey_data_with_clusters['mental_health_negative_impact_career'] == "No, it has not", 'mental_health_negative_impact_career'] = 5
survey_data_with_clusters.loc[survey_data_with_clusters['mental_health_negative_impact_career'] == "Yes, I think it would", 'mental_health_negative_impact_career'] = 0
survey_data_with_clusters.loc[survey_data_with_clusters['mental_health_negative_impact_career'] == "Yes, it has", 'mental_health_negative_impact_career'] = 0
# mental_health_negative_view_coworkersmental_health_negative_view_coworkers
survey_data_with_clusters.loc[survey_data_with_clusters['mental_health_negative_view_coworkers'] == "No, I don't think they would", 'mental_health_negative_view_coworkers'] = 5
survey_data_with_clusters.loc[survey_data_with_clusters['mental_health_negative_view_coworkers'] == "Maybe", 'mental_health_negative_view_coworkers'] = 1
survey_data_with_clusters.loc[survey_data_with_clusters['mental_health_negative_view_coworkers'] == "No, they do not", 'mental_health_negative_view_coworkers'] = 5
survey_data_with_clusters.loc[survey_data_with_clusters['mental_health_negative_view_coworkers'] == "Yes, I think they would", 'mental_health_negative_view_coworkers'] = 0
survey_data_with_clusters.loc[survey_data_with_clusters['mental_health_negative_view_coworkers'] == "Yes, they do", 'mental_health_negative_view_coworkers'] = 0
# mental_health_disorder_current
survey_data_with_clusters.loc[survey_data_with_clusters['mental_health_disorder_current'] == "Yes", 'mental_health_disorder_current'] = 0
survey_data_with_clusters.loc[survey_data_with_clusters['mental_health_disorder_current'] == "Maybe", 'mental_health_disorder_current'] = 0 # Thinking having one is as bad as having one for sure
survey_data_with_clusters.loc[survey_data_with_clusters['mental_health_disorder_current'] == "No", 'mental_health_disorder_current'] = 5 # Because of rating

C:\Users\SHEKHAR\AppData\Local\Temp\ipykernel_7680\3149824127.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\SHEKHAR\AppData\Local\Temp\ipykernel_7680\3149824127.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\SHEKHAR\AppData\Local\Temp\ipykernel_7680\3149824127.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\SHEKHAR\AppData\Local\Temp\ipykernel_7680\3149824127.py:9: SettingWithCopyWarning:


A v

In [69]:
for col in survey_data_with_clusters:
    print(str(col) + " " + str(survey_data_with_clusters[col].unique()))


company_mental_health [2 0 5]
company_mental_health_discussion [0 5]
company_mental_health_resources [0 5]
company_mental_health_anonymity [2 5 0]
company_mental_health_leave [4 3 2 0 1]
company_mental_health_options_awareness [0 5 2]
company_mental_health_discussion_impact [5 1 0]
mental_health_coworkers_discussion [3 5 0]
mental_health_supervisor_discussion [5 3 0]
company_mental_health_serious [2 5 0]
company_mental_health_coworker_discussion_negative_impact [5 0]
previous_and_current_company_mental_health_bad_response_experienced [5 1 0]
mental_health_negative_impact_career [1 5 0]
mental_health_negative_view_coworkers [5 1 0]
mental_health_disorder_current [5 0]


In [70]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
import numpy as np
X = survey_data_with_clusters.copy()
scaler = MinMaxScaler()
scaler.fit(X)
X=scaler.transform(X)
inertia = []
for i in range(1,11):
    kmeans = KMeans(
        n_clusters=i, init="k-means++",
        n_init=10,
        tol=1e-04, random_state=42
    )
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)
fig = go.Figure(data=go.Scatter(x=np.arange(1,11),y=inertia))
fig.update_layout(title="Inertia vs Cluster Number",xaxis=dict(range=[0,11],title="Cluster Number"),
                  yaxis={'title':'Inertia'},
                 annotations=[
        dict(
            x=3,
            y=inertia[2],
            xref="x",
            yref="y",
            text="Elbow!",
            showarrow=True,
            arrowhead=7,
            ax=20,
            ay=-40
        )
    ])


In [57]:
import plotly.express as px
kmeans = KMeans(
        n_clusters=3, init="k-means++",
        n_init=10,
        tol=1e-04, random_state=42
    )
kmeans.fit(X)


clusters=pd.DataFrame(X,columns=survey_data_with_clusters.columns)
clusters['label']=kmeans.labels_
polar=clusters.groupby("label").mean().reset_index()
polar=pd.melt(polar,id_vars=["label"])
fig4 = px.line_polar(polar, r="value", theta="variable", color="label", line_close=True,height=600,width=1200)
fig4.show()